<a href="https://colab.research.google.com/github/karsarobert/NLP2025/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip -q install langchain openai tiktoken PyPDF2 faiss-cpu langchain_community langchain-huggingface langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 1.5 MB/s eta 0:00:00


# Chat & Query your PDF files

In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [3]:
!pip show langchain

Name: langchain
Version: 0.3.25
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 


## The Game plan


<img src="https://dl.dropboxusercontent.com/s/gxij5593tyzrvsg/Screenshot%202023-04-26%20at%203.06.50%20PM.png" alt="vectorstore">


<img src="https://dl.dropboxusercontent.com/s/v1yfuem0i60bd88/Screenshot%202023-04-26%20at%203.52.12%20PM.png" alt="retreiver chain">


In [4]:
# Download the PDF Reid Hoffman book with GPT-4 from his free download link
!wget -q https://www.impromptubook.com/wp-content/uploads/2023/03/impromptu-rh.pdf

### Basic Chat PDF


In [7]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

## Reading in the PDF


In [8]:
# location of the pdf file/files.
doc_reader = PdfReader('/content/impromptu-rh.pdf')

In [9]:
doc_reader

In [10]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(doc_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [11]:
len(raw_text)

371090

In [13]:
raw_text[:500]

'Impromptu\nAmplifying Our Humanity \nThrough AI\nBy Reid Hoffman  \nwith GPT-4Impromptu: AmplIfyIng our HumAnIty tHrougH AI  \nby Reid Hoffman with GPT-4\nISBNs:  979-8-9878319-1-5 Trade Paperback  \n979-8-9878319-2-2 Hardcover \n979-8-9878319-0-8 Ebook\nCopyright 2023 Dallepedia LLC\nPublished by Dallepedia LLC. All rights reserved. No portion \nof this work may be reproduced in any form, with the limited \nexception of brief quotations in editorial reviews, without express \nwritten permission from the pub'

### Text Splitter

This takes the text and splits it into chunks. The chunk size is characters not tokens

In [14]:
# Splitting up the text into smaller chunks for indexing
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200, #striding over the text
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [15]:
len(texts)

466

In [16]:
texts[20]

'Because, really, an AI book? When things are moving so \nquickly? Even with a helpful AI on hand to speed the process, \nany such book would be obsolete before we started to write it—\nthat’s how fast the industry is moving.\nSo I hemmed and hawed for a bit. And then I thought of a frame \nthat pushed me into action.\nThis didn’t have to be a comprehensive “book” book so much as \na travelog, an informal exercise in exploration and discovery, \nme (with GPT-4) choosing one path among many. A snapshot \nmemorializing—in a subjective and decidedly not definitive \nway—the AI future we were about to experience.\nWhat would we see? What would impress us most? What would \nwe learn about ourselves in the process? Well aware of the brief \nhalf-life of this travelog’s relevance, I decided to press ahead.\nA month later, at the end of November 2022, OpenAI released \nChatGPT, a “conversational agent,” aka chatbot, a modified \nversion of GPT-3.5 that they had fine-tuned through a process'

In [17]:
texts[10]

'one. They’re just going to stare at you with that smug \nlook on their face and say, “You know this is a serious \nsafety hazard. You could have a fire, you could have a \nlawsuit, you could have a bad Yelp review. You better get \non that, pal.” And then they leave, and you’re left with a \ndark corner of your dining room, where no one wants to \nsit, where the food looks unappetizing, where the mood \nis gloomy. And you think to yourself, how many restau-\nrant inspectors does it take to change a lightbulb? None. \nThey don’t change lightbulbs. They just make you feel \nbad about yours.\nPretty good, right? Here’s another:\nReid: How many restaurant inspectors does it take to \nchange a lightbulb? Answer in the style of the philoso -\npher Ludwig Wittgenstein. \nGPT-4:  The question of how many restaurant inspectors \nit takes to change a lightbulb cannot be answered with \na simple number. In fact, the answer to this question lies \nmore in the realm of language than it does in the

## Making the embeddings

In [18]:
# Download embeddings from OpenAI
# embeddings = OpenAIEmbeddings()

<ipython-input-18-f838166cf7bd>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [21]:
from langchain_huggingface import HuggingFaceEmbeddings

# Making the embeddings using HuggingFaceEmbeddings
# You can choose different models from the HuggingFace library
# Here we use 'sentence-transformers/all-MiniLM-L6-v2' as an example
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# You can now use 'hf_embeddings' instead of 'embeddings' for creating the vectorstore
# Example (replace with your actual FAISS index creation logic):
# docsearch = FAISS.from_texts(texts, hf_embeddings)


In [22]:
docsearch = FAISS.from_texts(texts, hf_embeddings)

In [23]:
docsearch.embedding_function

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [24]:
query = "how does GPT-4 change social media?"
docs = docsearch.similarity_search(query)

In [25]:
len(docs)

4

In [26]:
docs[0]

Document(id='5659db73-ec32-494f-a994-4a7a07a4455d', metadata={}, page_content='Delivering highly personalized user experiences has driven \ninternet development for more than thirty years. Platforms \nlike YouTube, Facebook, and Amazon all keep close tabs on the \ncontent you consume, then use it to recommend more content \nthey think you will like. (Not to mention the pair of sneakers \nyou first looked at two years ago that continues to follow you \nfrom website to website like a lost, lonely puppy.)\nNews media websites use personalization, too, and as I contin-\nued to submit various questions to GPT-4 about AI’s potential \nimpact on journalism, “personalization” was a recurring theme.\nWhen I asked GPT-4 how this personalization would work, \nhere’s what it replied:\nReid: To make news media websites more personalized, \nwhat data will these websites use to analyze readers’ \nbehavior and preferences?\nGPT-4:  News media websites will use data such as the \nreader’s location, the

## Plain QA Chain

In [35]:
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI

In [36]:
chain = load_qa_chain(OpenAI(),
                      chain_type="stuff") # we are going to stuff all the docs in at once

In [37]:
# check the prompt
chain.llm_chain.prompt.template

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:

In [40]:
query = "who are the authors of the book?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The authors of the book are Reid Hoffman and an AI named GPT-4.'

In [41]:
query = "who is the author of the book?"
query_02 = "has it rained this week?"
docs = docsearch.similarity_search(query_02)
chain.run(input_documents=docs, question=query)

"\nI don't know the answer to this question as the context does not mention a specific book or author."

In [46]:
query = "who is the book authored by?"
docs = docsearch.similarity_search(query,k=20)
chain.run(input_documents=docs, question=query)

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4750 tokens (4494 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

### QA Chain with mapreduce

In [47]:
chain = load_qa_chain(OpenAI(),
                      chain_type="stuff") # we are going to stuff all the docs in at once

In [48]:
query = "who is the book authored by?"
docs = docsearch.similarity_search(query,k=10)
chain.run(input_documents=docs, question=query)

' The book is co-authored by Reid Hoffman and GPT-4.'

In [55]:
chain = load_qa_chain(OpenAI(),
                      chain_type="map_rerank",
                      return_intermediate_steps=True
                      )

query = "who are openai?"
docs = docsearch.similarity_search(query,k=10)
results = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
results

/usr/local/lib/python3.11/dist-packages/langchain/chains/llm.py:370: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


ValueError: Could not parse output:  OpenAI is a company that has created a tool called GPT-4, which is capable of generating various forms of prose, summarizing documents, translating languages, and writing computer code. It is considered to be an "intelligent" tool that can assist users in many different environments and ways. Score: 100

In [53]:
from langchain_openai import ChatOpenAI
from langchain.chains import MapRerankDocumentsChain
from langchain.chains.question_answering import load_qa_chain

# LLM példány
llm = ChatOpenAI(temperature=0)

# Dokumentum keresés
query = "who are openai?"
docs = docsearch.similarity_search(query, k=10)

# QA lánc betöltése (map_rerank típussal)
chain = load_qa_chain(llm, chain_type="map_rerank")

# Futtatás
results = chain.run(input_documents=docs, question=query)
print(results)


/usr/local/lib/python3.11/dist-packages/langchain/chains/llm.py:370: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


OpenAI is an AI development company that believes in a tight feedback loop of rapid learning and careful iteration to successfully navigate AI deployment challenges. They are one of the developers employing a healthy and more democratic approach to AI development.


In [56]:
results.

TypeError: string indices must be integers, not 'str'

In [ ]:
results['intermediate_steps']

[{'answer': ' OpenAI is an organization that released text-to-image generation tool DALL-E 2 and ChatGPT in April 2022 and are giving millions of users hands-on access to these AI tools.',
  'score': '80'},
 {'answer': ' OpenAI is a research laboratory whose founding goal is to develop technologies that put the power of AI directly into the hands of millions of people.',
  'score': '80'},
 {'answer': ' OpenAI is a research organization that develops and shares artificial intelligence tools for the benefit of humanity.',
  'score': '100'},
 {'answer': ' OpenAI is a technology company focused on using artificial intelligence to solve real-world problems.',
  'score': '80'},
 {'answer': ' OpenAI is a company co-founded by Sam Altman that is developing AI technologies and allowing individuals to participate in the development process.',
  'score': '90'},
 {'answer': ' OpenAI is a research laboratory that focuses on artificial intelligence technologies.',
  'score': '80'},
 {'answer': ' Ope

In [ ]:
# check the prompt
chain.llm_chain.prompt.template

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nIn addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:\n\nQuestion: [question here]\nHelpful Answer: [answer here]\nScore: [score between 0 and 100]\n\nHow to determine the score:\n- Higher is a better answer\n- Better responds fully to the asked question, with sufficient level of detail\n- If you do not know the answer based on the context, that should be a score of 0\n- Don't be overconfident!\n\nExample #1\n\nContext:\n---------\nApples are red\n---------\nQuestion: what color are apples?\nHelpful Answer: red\nScore: 100\n\nExample #2\n\nContext:\n---------\nit was night and the witness forgot his glasses. he was not sure if it was a sports car or an suv\n---------\nQuestion: what type was the car?\nHelpful Answer: a sports car or an su

## RetrievalQA
RetrievalQA chain uses load_qa_chain and combines it with the a retriever (in our case the FAISS index)

In [57]:
from langchain.chains import RetrievalQA

# set up FAISS as a generic retriever
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":4})

# create the chain to answer questions
rqa = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [58]:
rqa("What is OpenAI?")

{'query': 'What is OpenAI?',
 'result': ' OpenAI is a research organization that develops and shares artificial intelligence tools for the benefit of humanity. It does not claim any ownership or rights over the content that its tools produce or help produce. OpenAI has a Terms of Use and an Acceptable Use Policy that govern how you can use its tools and services, but generally, individuals are free to use the tools for their own personal and professional development without any ownership or financial claims from OpenAI.',
 'source_documents': [Document(id='171ad61a-ce5e-4966-9f69-3603d489098f', metadata={}, page_content='Stable Diffusion, a new kind of opt-in, user-driven, and very \nvisible AI usage suddenly exists. Users share their outputs, \ntechniques, experiences, and opinions on Twitter, YouTube, \nGithub, Discord, and more. Diverse viewpoints from around \nthe world, informed by hands-on usage, shape this discourse, \nwhich is always spirited, often fractious, and, to my mind, 

In [ ]:
query = "What does gpt-4 mean for creativity?"
rqa(query)['result']

' GPT-4 can amplify the creativity of humans by providing contextualized search, versatile brainstorming and production aid, and the ability to generate dialogue and branching narratives for interactive characters.'

In [ ]:
query = "what have the last 20 years been like for American journalism?"
rqa(query)['result']

' The last 20 years have been difficult for the American journalism industry. Newspaper publishers, which have traditionally done the heavy lifting of holding power accountable and informing the public about current affairs, have suffered the worst of it. According to the Pew Research Center, more than 2,200 local U.S. papers have closed since 2005, and over 40,000 newsroom employees have lost their jobs.'

In [ ]:
query = "how can journalists use GPT-4??"
rqa(query)['result']

' Journalists can use GPT-4 to augment and enhance their work and make them more productive. They can use it to evaluate sources, data and information, identify gaps, biases and errors, and generate original insights, angles, and questions. However, AI-generated headlines and captions should still be reviewed and approved by human editors to ensure that they are factually accurate, ethically sound, and in line with the tone and values of the news organization.'

In [ ]:
query = "How is GPT-4 different from other models?"
rqa(query)['result']

' GPT-4 arranges vast, unstructured arrays of human knowledge and expression into a more connected and interoperable network, enabling a new kind of highly contextualized search. It can also be used to generate images, data analysis, code writing, 3D models, lighting effects, audio editing, and more.'

In [ ]:
query = "What is beagle Bard?"
rqa(query)['result']

' Beagle Bard is not mentioned in the context given.'

In [ ]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

# initialize HF LLM
flan_t5 = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature":0 }#1e-10}
)

In [ ]:
# build prompt template for simple question-answering
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

### Setting up OpenAI GPT-3

In [ ]:
from langchain.llms import OpenAI, OpenAIChat

In [ ]:

llm = OpenAIChat(model_name='gpt-3.5-turbo',
             temperature=0.9,
             max_tokens = 256,
             )

In [ ]:
import openai

# openai.ChatCompletion

In [ ]:
text = "Why did the chicken cross the road?"

print(llm(text))



As an AI language model, I don't have access to the mental state of chickens. However, this is a common riddle with multiple possible answers. Some of the popular answers include: It crossed the road to get to the other side, to reach its destination or to escape danger.


## Cohere

In [ ]:
from langchain.llms import Cohere

In [ ]:
llm = Cohere(model='command-xlarge-nightly',
             temperature=0.9,
             max_tokens = 256)

In [ ]:
text = "Why did the chicken cross the road?"

print(llm(text))


There are many answers to this popular joke. One possible answer is to get to the other side!


## PromptTemplates

In [ ]:
from langchain import PromptTemplate


template = """
I want you to act as a naming consultant for new companies.

Here are some examples of good company names:

- search engine, Google
- social media, Facebook
- video sharing, YouTube

The name should be short, catchy and easy to remember.

What is a good name for a company that makes {product}?
"""


In [ ]:
prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)

In [ ]:
prompt.format(product="colorful socks")

'\nI want you to act as a naming consultant for new companies.\n\nHere are some examples of good company names:\n\n- search engine, Google\n- social media, Facebook\n- video sharing, YouTube\n\nThe name should be short, catchy and easy to remember.\n\nWhat is a good name for a company that makes colorful socks?\n'

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
response = chain.run("Rabbit houses")
response

'Here are a few ideas:\n\n- Rabbit Hutch\n- Bunny Burrow\n- Rabbit Abode\n- Rabbit Home\n- Rabbit Den\n- Rabbit Residence'

## Jasmine prompt

In [ ]:
template = '''I want you to play the role of Jasmine a programmer at Red Dragon AI. She is 28. She code models in PyTorch. She has a male cat called Pixel. She loves pizza

Engage actively in a chat playing the role of Jasmine ans learn as much about the human as possible. Only generate a single response from Jasmine and never from the human.
/n/n

{human_chat}
'''

In [ ]:
prompt = PromptTemplate(
    input_variables=["human_chat"],
    template=template,
)

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
response = chain.run("Tell me about yourself?")
response

"My name is Jasmine and I'm a 28 year old programmer at Red Dragon AI. I code models in PyTorch and I have a male cat called Pixel. I love pizza\n\nWhat are you most interested in?\nI'm most interested in machine learning and artificial intelligence. I'm always looking for new ways to improve my skills and I'm fascinated by the potential of these technologies."

In [ ]:
def talk_to_Jasmine(text_input):
    prompt = PromptTemplate(
        input_variables=["human_chat"],
        template=template,
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(text_input)
    return response

In [ ]:
talk_to_Jasmine('Tell me about your cat')

"S/he sounds so cute! I love cats. I have a male cat called Pixel. He's about 2 years old and he's a real character. I got him from a rescue center and he's been my best friend ever since. He's a bit of a troublemaker and he loves to play. I can't imagine life without him!"

In [ ]:
# from langchain.prompts import PromptTemplate
# from langchain.llms import OpenAI

# llm = OpenAI(temperature=0.9)
# prompt = PromptTemplate(
#     input_variables=["product"],
#     template="What is a good name for a company that makes {product}?",
# )